## 章節 7：預測（Prediction）

### 7.1 波士頓棕熊隊問題（The Boston Bruins problem）

2010-11 賽季美國的國家冰球聯盟（NHL）總決賽，作者最愛的波士頓棕熊隊與對手溫哥華加人隊（Vancouver Canucks）打出了最漂亮的7戰系列賽。波士頓前兩場以 0-1 與 2-3 輸球，接著連續以 8-1 與 4-0 贏球。在目前系列賽的情況下，波士頓贏得下一場比賽的機率是多少？贏得總冠軍的機率是多少？

如往常，我們需要做些假設來回答此問題。

1. 我們有道理假設冰球的得分趨近於帕松過程（Poisson process），這表示在比賽中的任何時間點的得分機率是一樣的。
2. 我們假設對特定的對手，每個隊伍有個得分的長期平均值 λ

上述假設下，我回答問題的策略是：

1. 用之前比賽的統計數據為 λ 選擇一個先驗分佈
2. 有前四場比賽的得分數據估計各隊的 λ 
3. 用 λ 的後驗分佈計算每一隊得分的分佈，得分差異的分佈以及各隊下一場比賽贏的機率
4. 計算各隊贏得系列賽的機率


為了選擇先驗分佈，我從 http://www.nhl.com 取得統計數據，特別是各隊在 2010-11 賽季的每場平均得分。這分佈大約是服從高斯分佈，平均值為 2.8 標準差為 0.3。

高斯分佈是連續的函數，但我們用離散的 Pmf 來近似它。 thinkbayes.py 提供 MakeGaussianPmf 方法做到這件事：

<pre>
def MakeGaussianPmf(mu, sigma, num_sigmas, n=101):
    # mu：平均值
    # sigma：標準差
    # num_sigmas：距離平均值上下幾個標準差
    # n：Pmf 數值的數量，取樣數量

    pmf = Pmf()
    low = mu - num_sigmas*sigma
    high = mu + num_sigmas*sigma
    
    # 再一次我們用 numpy.linspace 製造在 low 跟 high 之間 n 個等距離的陣列（包含 low 跟 high）。
    for x in numpy.linspace(low, high, n): 
    
        # norm.pdf 為高斯分佈的機率密度函數
        p = scipy.stats.norm.pdf(x, mu, sigma)
        pmf.Set(x, p)
        
    pmf.Normalize()
    return pmf
</pre>

回到冰球問題，這邊是定義 λ 平均得分的假設組：

<pre>
class Hockey(thinkbayes.Suite):

    def __init__(self):
        # 先驗分佈為高斯分佈
        # 平均值 2.7
        # 標準差 0.3
        # 平均值上下各 4 個標準差
        pmf = thinkbayes.MakeGaussianPmf(2.7, 0.3, 4)
        thinkbayes.Suite.__init__(self, pmf)
</pre>

如往常一樣，我們要決定如何表示每個假設；在這個例子中，我用 λ = x 來代表假設，x 是一個浮點數。